# Проект по SQL

- Целью проекта является исследования книжной базы сервисa для чтения книг по подписке.
- Задача - проанализировать базу данных c целью сформулировать ценностное предложение для нового продукта.
- В распоряжении база данных, содержащей пять таблиц с данными.

<a id='section_id'></a>
# Содержание

1. [Обзор исходных данных и заданий](#1)
2. [Загрузка библиотек и настройка соединения](#2)
3. [Задание №1](#3)   
4. [Задание №2](#4) 
5. [Задание №3](#5) 
6. [Задание №4](#6) 
7. [Задание №5](#7) 
8. [Вывод](#8) 

<a id='1'></a>
## 1. Обзор исходных данных и задач
[Назад к содержанию](#section_id)

Исходные данные

Таблица `books` - содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.


Таблица `authors` - Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

Таблица `publishers` - Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица `ratings` - Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица `reviews` - Содержит данные о пользовательских обзорах на книги:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.


**Задачи проекта**
 1. Подсчет количества книг, вышедших после 1 января 2000 года;
 2. Подсчет для каждой книги количества обзоров и среднюю оценку;
 3. Определение издательства, которое выпустило наибольшее число книг толще 50 страниц — исключатся из анализа брошюры;
 4. Определение автора с самой высокой средней оценкой книг — учет только книг с 50 и более оценками;
 5. Подсчет среднего количества обзоров от пользователей, которые поставили больше 48 оценок.

## 2. Загрузка библиотек и настройка соединения с БД
<a id='2'></a>
[Назад к содержанию](#section_id)

In [2]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

In [3]:
# устанавливаем параметры
db_config = {
    'user': '...', # имя пользователя
    'pwd': '...', # пароль
    'host': '...',
    'port': ..., # порт подключения
    'db': '...' # название базы данных
}  
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

query = '''
SELECT *,COUNT(*) OVER() AS table_size
FROM books 
LIMIT 5 
'''
query_1 = '''
SELECT *,COUNT(*) OVER() AS table_size
FROM ratings
LIMIT 5 
'''
query_2 = '''
SELECT *,COUNT(*) OVER() AS table_size
FROM authors
LIMIT 5 
'''
query_3 = '''
SELECT *,COUNT(*) OVER() AS table_size
FROM reviews
LIMIT 5 
'''
query_4 = '''
SELECT *,COUNT(*) OVER() AS table_size
FROM publishers
LIMIT 5 
'''
# Выгружаем таблицы
books = pd.read_sql_query(query, con=engine) 
ratings = pd.read_sql_query(query_1, con=engine) 
authors = pd.read_sql_query(query_2, con=engine) 
reviews = pd.read_sql_query(query_3, con=engine) 
publishers = pd.read_sql_query(query_4, con=engine) 

In [3]:
# Отобразим каждую таблицу и количество строк в ней:

d = [books,
     ratings,
     authors,
     reviews,
     publishers]
for df in d:
    display(df)

,book_id,author_id,title,num_pages,publication_date,publisher_id,table_size
0,1,546,'Salem's Lot,594,2005-11-01,93,1000
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336,1000
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135,1000
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309,1000
4,5,125,1776,386,2006-07-04,268,1000


,rating_id,book_id,username,rating,table_size
0,1,1,ryanfranco,4,6456
1,2,1,grantpatricia,2,6456
2,3,1,brandtandrea,5,6456
3,4,2,lorichen,3,6456
4,5,2,mariokeller,2,6456


,author_id,author,table_size
0,1,A.S. Byatt,636
1,2,Aesop/Laura Harris/Laura Gibbs,636
2,3,Agatha Christie,636
3,4,Alan Brennert,636
4,5,Alan Moore/David Lloyd,636


,review_id,book_id,username,text,table_size
0,1,1,brandtandrea,Mention society tell send professor analysis. ...,2793
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...,2793
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...,2793
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...,2793
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...,2793


,publisher_id,publisher,table_size
0,1,Ace,340
1,2,Ace Book,340
2,3,Ace Books,340
3,4,Ace Hardcover,340
4,5,Addison Wesley Publishing Company,340


## 3. Задание № 1
<a id='3'></a>
[Назад к содержанию](#section_id)

Необходимо подсчитать, сколько книг вышло после 1 января 2000 года.

In [4]:
query = '''
SELECT COUNT(book_id) as "Count of books"
FROM books
WHERE publication_date > '2000-01-01'

'''
df_1 = pd.read_sql_query(query, con=engine) 
df_1

,Count of books
0,819


**Вывод:** После 1 января 2000 года вышло 819 книг.

## 4. Задание № 2
<a id='4'></a>
[Назад к содержанию](#section_id)

Необходимо подсчитать для каждой книги  количество обзоров и среднюю оценку.

In [5]:
# напишите запрос
query = '''
SELECT b.title, 
       b.book_id, 
       COUNT(DISTINCT (review_id)) as review_count, 
       ROUND((AVG (rating)),2) as avg_rating
FROM books as b
LEFT JOIN reviews re ON re.book_id = b.book_id
LEFT JOIN ratings ra ON ra.book_id = b.book_id
GROUP BY b.book_id
ORDER BY COUNT(review_id) DESC, AVG (rating) DESC

'''
df_2 = pd.read_sql_query(query, con=engine) 
df_2
# выполните запрос

,title,book_id,review_count,avg_rating
0,Twilight (Twilight #1),948,7,3.66
1,The Hobbit or There and Back Again,750,6,4.13
2,The Catcher in the Rye,673,6,3.83
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.41
4,Harry Potter and the Chamber of Secrets (Harry...,299,6,4.29
...,...,...,...,...
995,Leonardo's Notebooks,387,0,4.00
996,Essential Tales and Poems,221,0,4.00
997,Disney's Beauty and the Beast (A Little Golden...,191,0,4.00
998,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.67


**Вывод:**

При сортировке по количеству обзоров на книги, топ-5 следующий
1. Twilight (Twilight #1)	
2. The Hobbit or There and Back Again	
3. The Catcher in the Rye	
4. Harry Potter and the Prisoner of Azkaban 
5. Harry Potter and the Chamber of Secrets 

## 5. Задание № 3
<a id='5'></a>
[Назад к содержанию](#section_id)

Необходимо определить издательство, которое выпустило наибольшее число книг толще
50 страниц.

In [8]:
# напишите запрос
query = '''

SELECT ps.publisher, COUNT (b.book_id) as books_count
FROM publishers ps
JOIN books as b ON b.publisher_id = ps.publisher_id
WHERE ps.publisher_id IN (
    COALESCE((
        SELECT b.publisher_id
        FROM books as b
        WHERE b.num_pages > 50
        GROUP BY b.publisher_id 
        ORDER BY COUNT(book_id) DESC
        LIMIT 1
    ), 0)
)
GROUP BY ps.publisher
ORDER BY COUNT (b.book_id) DESC

'''
df_4 = pd.read_sql_query(query, con=engine) 
df_4
# выполните запрос

,publisher,books_count
0,Penguin Books,42


**Вывод:** Издательство Penguin Books выпустило самое большое количество книг толще 50 страниц - 42

## 6. Задание № 4
<a id='6'></a>
[Назад к содержанию](#section_id)

Необходимо определить автора с самой высокой средней оценкой книг, учитывать
только книги с 50 и более оценками

In [9]:
query = '''
SELECT a.author, ROUND((AVG (ra.rating)),2) as rating
FROM authors as a
JOIN books b ON b.author_id = a.author_id
JOIN ratings ra ON ra.book_id = b.book_id
JOIN (
    SELECT ra.book_id
    FROM ratings as ra
    GROUP BY ra.book_id
    HAVING COUNT (rating_id) >= 50
) AS books_with_more_than_50_ratings
ON books_with_more_than_50_ratings.book_id = b.book_id
GROUP BY a.author_id
ORDER BY AVG (ra.rating) DESC
LIMIT 1

'''
df_4 = pd.read_sql_query(query, con=engine) 
df_4

,author,rating
0,J.K. Rowling/Mary GrandPré,4.29


**Вывод:**
Автором с самой высокой средней оценкой книг является J.K. Rowling/Mary GrandPré c оценкой 4.29

## 7. Задача № 5
<a id='7'></a>
[Назад к содержанию](#section_id)

Необходимо посчитать среднее количество обзоров от пользователей, которые поставили
больше 48 оценок.

In [11]:
query = '''
WITH
    ratings_subquery AS (
        SELECT ra.username,
               COUNT(ra.rating) AS ratings_quantity
        FROM ratings ra
        GROUP BY ra.username
        HAVING COUNT(ra.rating) > 48),
    reviews_subquery AS ( SELECT re.username,
                          COUNT(re.text) AS reviews_quantity
                          FROM reviews re
                          GROUP BY re.username)
SELECT ROUND(AVG(reviews_quantity)) avg_count
FROM ratings_subquery
JOIN reviews_subquery ON reviews_subquery.username = ratings_subquery.username
'''
df_5 = pd.read_sql_query(query, con=engine) 
df_5

,avg_count
0,24.0


**Вывод**: Пользователи, поставившие более 48 оценок, в среднем оставляют 24 обзора.

## 8. Общий вывод
<a id='8'></a>
[Назад к содержанию](#section_id)

Было проведено исследование книжной базы данных.
На основе поставленных задач были написаны запросы.
По результатам запросов сформированы следующие выводы

- После 1 января 2000 года вышло 819 книг.
- Автором с самой высокой средней оценкой книг является J.K. Rowling/Mary GrandPré c оценкой 4.28 
- Пользователи, поставившие более 48 оценок, в среднем оставляют 24 обзора.
- Издательство Penguin Books выпустило самое большое количество книг толще 50 страниц - 42
- При сортировке по количеству обзоров на книги, топ-5 следующий
  1. Twilight (Twilight #1)
  2. The Hobbit or There and Back Again
  3. The Catcher in the Rye
  4. Harry Potter and the Prisoner of Azkaban
  5. Harry Potter and the Chamber of Secrets